In [ ]:
# import library
# please save figures in the folder named "figures"


- what to do
    - give raphs for each learning rate
    - the change of regret?, total payoff, decisions?
    - horizontal axis must be the number of rounds
    - vertical axis must be one of them.

# A. Adversarial Fair Payoff

# B. Bernoulli Payoff

# Data in the wild

# Adversarial generative model